In [1]:
import numpy as np
import pandas as pd
import shapely
import geopandas as gpd
from pathlib import Path

In [2]:
raw_data_dir = "raw_data"
processed_data_dir = "processed_data"

## Grid Geometry Data

In [3]:
df = gpd.read_file(f"{raw_data_dir}/supporting_data/grid/LAEI2019_Grid.shp")

df.head()

,Grid_ID_19,GridIdEx,1km2_ID,Easting,Northing,Area_km2,Borough,Zone,geometry
0,1,1,5910,510500,203500,1.0,NonGLA,NonGLA,"POLYGON ((511000.001 203000.001, 509999.999 20..."
1,2,2,5911,511500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512000.003 203000.001, 511000.001 20..."
2,3,3,5912,512500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512999.997 203000.001, 512000.003 20..."
3,4,4,5915,515500,203500,1.0,NonGLA,NonGLA,"POLYGON ((516000.004 203000.001, 515000.002 20..."
4,5,5,5916,516500,203500,1.0,NonGLA,NonGLA,"POLYGON ((516999.998 203000.001, 516000.004 20..."


In [4]:
df.rename(columns={
    "Grid_ID_19": "grid_id",
    "Borough": "borough",
    "Zone": "zone",
    "Area_km2": "area",
}, inplace=True)

df.head()

,grid_id,GridIdEx,1km2_ID,Easting,Northing,area,borough,zone,geometry
0,1,1,5910,510500,203500,1.0,NonGLA,NonGLA,"POLYGON ((511000.001 203000.001, 509999.999 20..."
1,2,2,5911,511500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512000.003 203000.001, 511000.001 20..."
2,3,3,5912,512500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512999.997 203000.001, 512000.003 20..."
3,4,4,5915,515500,203500,1.0,NonGLA,NonGLA,"POLYGON ((516000.004 203000.001, 515000.002 20..."
4,5,5,5916,516500,203500,1.0,NonGLA,NonGLA,"POLYGON ((516999.998 203000.001, 516000.004 20..."


## Emission Data

In [5]:
emissions_summary_path = Path(f"{raw_data_dir}/emissions_summary")

in_scope_dfs = [df]
for path in emissions_summary_path.rglob("*-all-sources.shp"):
    print(f"{str(path).split("/")[-1]}")

    emissions_df = gpd.read_file(path, columns=("pollutant", "all_2019"))

    pollutant = emissions_df.iloc[0]["pollutant"]
    emissions_df.rename(columns={"all_2019": f"total_pollutant_{pollutant}_2019"},
                        inplace=True)

    in_scope_dfs.append(emissions_df.drop(columns=["pollutant", "geometry"]))

LAEI2019-co2-grid-emissions-all-sources.shp
LAEI2019-pm10-grid-emissions-all-sources.shp
LAEI2019-pm2-5-grid-emissions-all-sources.shp
LAEI2019-nox-grid-emissions-all-sources.shp


In [6]:
df = pd.concat(in_scope_dfs, axis=1)

df.head()

,grid_id,GridIdEx,1km2_ID,Easting,Northing,area,borough,zone,geometry,total_pollutant_co2_2019,total_pollutant_pm10_2019,total_pollutant_pm2.5_2019,total_pollutant_nox_2019
0,1,1,5910,510500,203500,1.0,NonGLA,NonGLA,"POLYGON ((511000.001 203000.001, 509999.999 20...",1212.883859,0.162653,0.121533,1.340824
1,2,2,5911,511500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512000.003 203000.001, 511000.001 20...",23432.691922,9.893812,2.349748,45.665479
2,3,3,5912,512500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512999.997 203000.001, 512000.003 20...",11388.566615,4.926730,1.125090,19.841723
3,4,4,5915,515500,203500,1.0,NonGLA,NonGLA,"POLYGON ((516000.004 203000.001, 515000.002 20...",2682.887565,0.485676,0.370099,5.732810
4,5,5,5916,516500,203500,1.0,NonGLA,NonGLA,"POLYGON ((516999.998 203000.001, 516000.004 20...",2353.626904,0.355503,0.270166,4.426324


## Rail Network 

In [7]:
gdf_1 = gpd.read_file(f"{raw_data_dir}/supporting_data/rail/LAEI_Rail_Network_Link_ExactCut.shp")

gdf_1.head(2)

,DESCRIPT,GRID_ID,GRID_ID0,REFERENCES,X_COORD,Y_COORD,BOROUGHNAM,F1KM_GRID,REFINED,Shape_Leng,LinkID,IDIntersec,geometry
0,Between London Charing Cross and London Waterl...,3266.0,9886.0,TQ3080,530500.0,180500.0,Lambeth,Central,Central,494.264332,1,1,"LINESTRING (530999.996 180060.288, 530966.983 ..."
1,Between London Charing Cross and London Waterl...,3289.0,9886.0,TQ3080,530500.0,180500.0,City of Westminster,Central,Central,284.378990,1,2,"LINESTRING (530553.809 180272.321, 530494.046 ..."


In [8]:
def process_links_groupby_func(gdf, fields=[]):
    crs = gdf.crs

    exploded_gdf = gdf.explode(index_parts=True)
    merged_geom = shapely.ops.linemerge(exploded_gdf.geometry.tolist())

    result = gpd.GeoDataFrame({"geometry": [merged_geom]}, crs=crs)

    for field in fields:
        result[field] = gdf[field].iloc[0]

    return result

In [9]:
compiled_gdf = gdf_1.groupby("LinkID").apply(process_links_groupby_func, 
                                             include_groups=False, 
                                             fields=["DESCRIPT"]).reset_index(drop=True)

compiled_gdf.head(2)

,geometry,DESCRIPT
0,"LINESTRING (531270.896 180037.924, 531206.782 ...",Between London Charing Cross and London Waterl...
1,"LINESTRING (532416.572 180115.002, 532319.338 ...",Between Metropolitan Jn and London Waterloo (E...


In [10]:
def total_intersecting_agg(df, df2, agg):
    df2 = df2.to_crs(df.crs)
    
    overlap = gpd.overlay(df1=df, df2=df2, how="intersection", keep_geom_type=False)
    aggregated_data = overlap.groupby("grid_id").agg(**agg).reset_index()

    df = df.merge(aggregated_data, how='left', on='grid_id')
    
    for column in agg.keys():
        df[column] = df[column].fillna(0.0).astype(float).apply(lambda x: round(x, 6))
        
    return df

In [11]:
agg_length_by_link = {f"total_rail_length": ("geometry", lambda x: np.sum(x.length))}

df = total_intersecting_agg(df, compiled_gdf, agg_length_by_link)

In [12]:
df.head(2)

,grid_id,GridIdEx,1km2_ID,Easting,Northing,area,borough,zone,geometry,total_pollutant_co2_2019,total_pollutant_pm10_2019,total_pollutant_pm2.5_2019,total_pollutant_nox_2019,total_rail_length
0,1,1,5910,510500,203500,1.0,NonGLA,NonGLA,"POLYGON ((511000.001 203000.001, 509999.999 20...",1212.883859,0.162653,0.121533,1.340824,0.0
1,2,2,5911,511500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512000.003 203000.001, 511000.001 20...",23432.691922,9.893812,2.349748,45.665479,0.0


In [13]:
compiled_gdf.geometry

0      LINESTRING (531270.896 180037.924, 531206.782 ...
1      LINESTRING (532416.572 180115.002, 532319.338 ...
2      LINESTRING (532596.784 180808.023, 532545.162 ...
3      LINESTRING (532915.497 180197.678, 532665.045 ...
4      LINESTRING (532665.045 180251.483, 532610.388 ...
                             ...                        
651    LINESTRING (522229.642 166132.934, 522216.669 ...
652    LINESTRING (522582.795 167750.381, 522544.958 ...
653    LINESTRING (536725.049 177075.27, 536604.399 1...
654    LINESTRING (536992.234 186446.929, 537000.001 ...
655    LINESTRING (556810.03 178011.084, 556249.168 1...
Name: geometry, Length: 656, dtype: geometry

In [14]:
# shapely.geometry.Point((compiled_gdf.geometry.iloc[100].coords[0]))

In [15]:
def get_start_point(geom):
    if geom.geom_type == 'LineString':
        return shapely.geometry.Point(geom.coords[0])
    elif geom.geom_type == 'MultiLineString':
        first_line = geom.geoms[0]  # Access first LineString
        return shapely.geometry.Point(first_line.coords[0])
    else:
        return None


def get_end_point(geom):
    if geom.geom_type == 'LineString':
        return shapely.geometry.Point(geom.coords[-1])
    elif geom.geom_type == 'MultiLineString':
        last_line = geom.geoms[-1]  # Access last LineString
        return shapely.geometry.Point(last_line.coords[-1])
    else:
        return None

start_points = compiled_gdf.geometry.apply(get_start_point)
end_points = compiled_gdf.geometry.apply(get_end_point)

In [16]:
all_points = pd.concat([start_points, end_points])
all_points_counts = all_points.value_counts()

all_points_counts

geometry
POINT (529300.293 176341.592)    6
POINT (526075.592 163864.174)    5
POINT (544401.535 184330.765)    5
POINT (537897.377 184459.628)    5
POINT (531799.998 172899.997)    5
                                ..
POINT (528613.653 178470.583)    1
POINT (503643.609 176668.568)    1
POINT (508033.848 201997.848)    1
POINT (528712.041 178042.145)    1
POINT (530091.019 182980.981)    1
Name: count, Length: 626, dtype: int64

In [17]:
all_points_gdf = gpd.GeoDataFrame(geometry=all_points)

all_points_gdf[f"total_rail_link_terminations"] = all_points_gdf.geometry.map(all_points_counts)

all_points_gdf = all_points_gdf.drop_duplicates(subset="geometry").reset_index(drop=True)

all_points_gdf.head()

,geometry,total_rail_link_terminations
0,POINT (531270.896 180037.924),2
1,POINT (532416.572 180115.002),4
2,POINT (532596.784 180808.023),2
3,POINT (532915.497 180197.678),4
4,POINT (532665.045 180251.483),2


In [18]:
agg_stations_by_link = {f"total_rail_stations": ("geometry", np.size),
                       f"total_rail_link_terminations": (f"total_rail_link_terminations", np.sum)}


In [19]:
df = total_intersecting_agg(df, all_points_gdf, agg_stations_by_link)

df.head(2)

/tmp/ipykernel_13235/2227677249.py:5: FutureWarning: The provided callable <function sum at 0x7de781bc6340> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  aggregated_data = overlap.groupby("grid_id").agg(**agg).reset_index()


,grid_id,GridIdEx,1km2_ID,Easting,Northing,area,borough,zone,geometry,total_pollutant_co2_2019,total_pollutant_pm10_2019,total_pollutant_pm2.5_2019,total_pollutant_nox_2019,total_rail_length,total_rail_stations,total_rail_link_terminations
0,1,1,5910,510500,203500,1.0,NonGLA,NonGLA,"POLYGON ((511000.001 203000.001, 509999.999 20...",1212.883859,0.162653,0.121533,1.340824,0.0,0.0,0.0
1,2,2,5911,511500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512000.003 203000.001, 511000.001 20...",23432.691922,9.893812,2.349748,45.665479,0.0,0.0,0.0


## Passenger Shipping Network 

In [20]:
gdf_2 = gpd.read_file(f"{raw_data_dir}/supporting_data/shipping/LAEI_PassengerShipping_Network_Link.shp")

compiled_gdf_2 = gdf_2.groupby("LINK_ID").apply(process_links_groupby_func, 
                                                include_groups=False, 
                                                fields=[]).reset_index(drop=True)

compiled_gdf_2.head(2)

,geometry
0,"LINESTRING (501983.375 171937.923, 502074.417 ..."
1,"LINESTRING (517734.921 169569.5, 517710.969 16..."


In [21]:
np.sum(compiled_gdf_2.geometry.iloc[0].length)

20694.576469259333

In [22]:
compiled_gdf_2 = compiled_gdf_2.to_crs(df.crs)

In [23]:
overlap = gpd.overlay(df1=df, df2=compiled_gdf_2, how="intersection", keep_geom_type=False)

In [24]:
aggregated_data_2 = overlap.groupby("grid_id")["geometry"].apply(lambda x: np.sum(x.length)).reset_index().rename({"geometry": 'total_shipping_length'}, axis=1)

In [25]:
aggregated_data_2

,grid_id,total_shipping_length
0,231,1252.399494
1,234,61.030845
2,235,1176.881381
3,236,756.920464
4,251,18.171025
...,...,...
176,3414,104.574233
177,3415,56.586628
178,3429,262.660613
179,3437,1003.999331


In [26]:
df = df.merge(aggregated_data_2, how='left', on='grid_id')

In [27]:
df["total_shipping_length"] = df["total_shipping_length"].fillna(0.0).astype(float).apply(lambda x: round(x, 6))

In [28]:
start_points = compiled_gdf_2.geometry.apply(get_start_point)
end_points = compiled_gdf_2.geometry.apply(get_end_point)

all_points = pd.concat([start_points, end_points])
all_points_counts = all_points.value_counts()

all_points_gdf_2 = gpd.GeoDataFrame(geometry=all_points)
all_points_gdf_2[f"total_shiping_link_terminations"] = all_points_gdf_2.geometry.map(all_points_counts)

all_points_gdf_2 = all_points_gdf_2.drop_duplicates(subset="geometry").reset_index(drop=True)

all_points_gdf_2.head(2)

,geometry,total_shiping_link_terminations
0,POINT (501983.375 171937.922),1
1,POINT (517734.921 169569.5),2


In [29]:
all_points_gdf_2 = all_points_gdf_2.to_crs(df.crs)

In [30]:
overlap_2 = gpd.overlay(df1=df, df2=all_points_gdf_2, how="intersection", keep_geom_type=False)

In [31]:
overlap_2[["grid_id", "total_shiping_link_terminations", "geometry"]]

,grid_id,total_shiping_link_terminations,geometry
0,251,1,POINT (501983.375 171937.922)
1,840,2,POINT (517566.087 174668.681)
2,940,2,POINT (515418.174 168481.832)
3,2052,2,POINT (517734.921 169569.5)
4,2432,1,POINT (543287.677 179384.591)
5,2433,2,POINT (543803.611 179605.947)
6,2837,1,POINT (536898.537 180319.773)
7,2838,2,POINT (536806.729 180277.005)
8,2838,1,POINT (536702.305 180241.486)
9,2859,2,POINT (538285.695 178094.63)


In [32]:
aggregated_data_22 = overlap_2.groupby("grid_id").agg(
    total_shipping_stations=("geometry", "size"),
    total_shipping_link_terminations=("total_shiping_link_terminations", "sum")
).reset_index()

aggregated_data_22.head(2)

,grid_id,total_shipping_stations,total_shipping_link_terminations
0,251,1,1
1,840,1,2


In [33]:
df = df.merge(aggregated_data_22, how='left', on='grid_id')
df["total_shipping_stations"] = df["total_shipping_stations"].fillna(0.0).astype(float).apply(lambda x: round(x, 6))
df["total_shipping_link_terminations"] = df["total_shipping_link_terminations"].fillna(0.0).astype(float).apply(lambda x: round(x, 6))

In [34]:
df_rail_ship = df.copy()

##  Adding Road Data

In [35]:
RELEVANT_ROAD_FEATURES = {
                        "Road Classification": "road_type",
                        " Speed (km/hr) - Except Buses ": "road_speed_non_bus",
                        " Speed (km/hr) - Buses Only ": "road_speed_bus",
                        " AADT Motorcycle ": "aadt_motorcycle",
                        " AADT Taxi ": "aadt_taxi",
                        " AADT Petrol Car ": "aadt_petrol_car",
                        " AADT Diesel Car ": "aadt_diesel_car",
                        " AADT Electric Car ": "aadt_electric_car",
                        " AADT Petrol PHV ": "aadt_petrol_phv",
                        " AADT Diesel PHV ": "aadt_diesel_phv",
                        " AADT Electric PHV ": "aadt_electric_phv",
                        " AADT Petrol LGV ": "aadt_petrol_lgv",
                        " AADT Diesel LGV ": "aadt_diesel_lgv",
                        " AADT Electric LGV ": "aadt_electric_lgv",
                        " AADT 2019 - HGVs - Rigid - 2 Axles ": "aadt_hgv_rigid_2_axles",
                        " AADT 2019 - HGVs - Rigid - 3 Axles ": "aadt_hgv_rigid_3_axles",
                        " AADT 2019 - HGVs - Rigid - 4 or more Axles ": "aadt_hgv_rigid_4_or_more_axles",
                        " AADT 2019 - HGVs - Articulated - 3 to 4 Axles ": "aadt_hgv_articulated_3_to_4_axles",
                        " AADT 2019 - HGVs - Articulated - 5 Axles ": "aadt_hgv_articulated_5_axles",
                        " AADT 2019 - HGVs - Articulated - 6 Axles ": "aadt_hgv_articulated_6_axles",
                        " AADT 2019 - Buses ": "aadt_bus",
                        " AADT 2019 - Coaches ": "aadt_coach",
                        " VKM Motorcycle ": "vkm_motorcycle",
                        " VKM Taxi ": "vkm_taxi",
                        " VKM Petrol Car ": "vkm_petrol_car",
                        " VKM Diesel Car ": "vkm_diesel_car",
                        " VKM Electric Car ": "vkm_electric_car",
                        " VKM Petrol PHV ": "vkm_petrol_phv",
                        " VKM Diesel PHV ": "vkm_diesel_phv",
                        " VKM Electric PHV ": "vkm_electric_phv",
                        " VKM Petrol LGV ": "vkm_petrol_lgv",
                        " VKM Diesel LGV ": "vkm_diesel_lgv",
                        " VKM Electric LGV ": "vkm_electric_lgv",
                        " VKM 2019 - HGVs - Rigid - 2 Axles ": "vkm_hgv_rigid_2_axles",
                        " VKM 2019 - HGVs - Rigid - 3 Axles ": "vkm_hgv_rigid_3_axles",
                        " VKM 2019 - HGVs - Rigid - 4 or more Axles ": "vkm_hgv_rigid_4_or_more_axles",
                        " VKM 2019 - HGVs - Articulated - 3 to 4 Axles ": "vkm_hgv_articulated_3_to_4_axles",
                        " VKM 2019 - HGVs - Articulated - 5 Axles ": "vkm_hgv_articulated_5_axles",
                        " VKM 2019 - HGVs - Articulated - 6 Axles ": "vkm_hgv_articulated_6_axles",
                        " VKM 2019 - Buses ": "vkm_bus",
                        " VKM 2019 - Coaches ": "vkm_coach",
                        }

In [36]:
gdf_r = gpd.read_file(f"{raw_data_dir}/supporting_data/road/shape/laei-2019-major-roads-final-unique-toids-flows-speeds-osgb.shp")
gdf_r.head(2)

,TOID,TOID_iBus,NewRdAdded,LAEIZone,Borough,Road_Class,F_Mcycle,F_Taxi,F_Car,F_PHV,F_LGV,F_HGV_Rigi,F_HGV_Arti,F_Bus,F_Coach,Tot_AADT19,Sp_kph,Sp_kph_Bus,Length_m,geometry
0,osgb4000000027947700,None,N,Non-GLA,NonGLA,A Road,112.0,15.0,9283.0,169.0,1117.0,286.0,340.0,0.0,16.0,11338.0,100.5,0.0,56.0,"LINESTRING (507713.001 201312, 507707.001 2013..."
1,osgb4000000027908760,None,N,Non-GLA,NonGLA,A Road,68.0,10.0,5175.0,622.0,831.0,64.0,76.0,0.0,27.0,6873.0,47.9,0.0,117.7,"LINESTRING (511479.001 200468, 511500.001 2004..."


In [37]:
r_df = pd.read_excel(f"{raw_data_dir}/supporting_data/road/excel/laei-2019-major-roads-vkm-flows-speeds.xlsx")
r_df.head(2)

,Year,TOID,LAEI Zone,Borough,Road Classification,AADT Motorcycle,AADT Taxi,AADT Petrol Car,AADT Diesel Car,AADT Electric Car,...,VKM Electric LGV,VKM 2019 - HGVs - Rigid - 2 Axles,VKM 2019 - HGVs - Rigid - 3 Axles,VKM 2019 - HGVs - Rigid - 4 or more Axles,VKM 2019 - HGVs - Articulated - 3 to 4 Axles,VKM 2019 - HGVs - Articulated - 5 Axles,VKM 2019 - HGVs - Articulated - 6 Axles,VKM 2019 - Buses,VKM 2019 - Coaches,VKM 2019 - Total
0,2019,osgb4000000027947700,Non-GLA,Non-GLA,A Road,112,15,5470,3780,32,...,50,3659,736,1451,531,2759,3659,-,327,231749
1,2019,osgb4000000027908760,Non-GLA,Non-GLA,A Road,68,10,3049,2107,18,...,78,1718,344,687,258,1289,1718,-,1160,295268


In [38]:
r_df.rename(columns=RELEVANT_ROAD_FEATURES, inplace=True)

In [39]:
gdf_r = gdf_r.merge(r_df, left_index=True, right_index=True)

In [40]:
non_numeric = ("geometry", "road_type")
numeric_cols = [col for col in gdf_r.columns if col not in non_numeric]

gdf_r[numeric_cols] = gdf_r[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

gdf_r = gdf_r.to_crs(df.crs)

In [41]:
road_type_lookup = {"A": "a_road",
                    "B": "b_road",
                    "C": "c_unclassified_road",
                    "M": "motorway"}

gdf_r["road_type"] = gdf_r["road_type"].apply(lambda x: "motorway" if x == "A1M" else road_type_lookup[x[0]])

gdf_r["toid_road_length"] = gdf_r.geometry.length

In [42]:
temp_df_r = df[["grid_id"]]

overlap_r = gpd.overlay(df1=df, df2=gdf_r, how="intersection", keep_geom_type=False)
overlap_r['length'] = overlap_r.geometry.length
overlap_r['ratio'] = overlap_r["length"] / overlap_r["toid_road_length"]

vkm_cols = [col for col in overlap_r.columns if col.startswith('vkm_')]
for col in vkm_cols:
    overlap_r[col] = overlap_r[col] * overlap_r['ratio']
    
aadt_cols = [col for col in overlap_r.columns if col.startswith('aadt_')]
overlap_r_by_grid_id = overlap_r.groupby("grid_id")
vkm_sums = overlap_r_by_grid_id[vkm_cols].sum()
aadt_sums = overlap_r_by_grid_id[aadt_cols].sum()

temp_df_r = temp_df_r.merge(vkm_sums, how='left', left_on='grid_id', right_index=True)
temp_df_r = temp_df_r.merge(aadt_sums, how='left', left_on='grid_id', right_index=True)

In [43]:
road_grouping = overlap_r.groupby(["grid_id", "road_type"])

road_lengths = road_grouping["length"].sum().reset_index()
road_lengths_pivot = road_lengths.pivot(index="grid_id", columns="road_type", values="length")

road_lengths_pivot.columns = [f'total_road_length_{col}' for col in road_lengths_pivot.columns]
temp_df_r = temp_df_r.merge(road_lengths_pivot, how='left', left_on='grid_id', right_index=True)

In [44]:
road_counts = road_grouping.size().reset_index(name="road_count")
road_counts_pivot = road_counts.pivot(index="grid_id", columns="road_type", values="road_count")

road_counts_pivot.columns = [f"total_road_count_{col}" for col in road_counts_pivot.columns]
temp_df_r = temp_df_r.merge(road_counts_pivot, how="left", left_on="grid_id", right_index=True)

In [45]:
aggregated_data_r = overlap_r.groupby("grid_id").agg(**{"mean_road_speed_non_bus": ("road_speed_non_bus", "mean"),
                                "mean_road_speed_bus": ("road_speed_bus", "mean")})

In [46]:
temp_df_r = temp_df_r.merge(aggregated_data_r, how='left', on='grid_id')
temp_df_r.fillna(value=0, inplace=True)

temp_df_r.head(2)

,grid_id,vkm_motorcycle,vkm_taxi,vkm_petrol_car,vkm_diesel_car,vkm_electric_car,vkm_petrol_phv,vkm_diesel_phv,vkm_electric_phv,vkm_petrol_lgv,...,total_road_length_a_road,total_road_length_b_road,total_road_length_c_unclassified_road,total_road_length_motorway,total_road_count_a_road,total_road_count_b_road,total_road_count_c_unclassified_road,total_road_count_motorway,mean_road_speed_non_bus,mean_road_speed_bus
0,1,0.00000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0
1,2,456307.55402,40493.744455,3.949915e+07,2.729787e+07,233904.426579,1.930342e+06,1.587845e+06,26936.065971,121945.680662,...,0.0,0.0,0.0,8143.46032,0.0,0.0,0.0,43.0,89.418605,0.0


In [47]:
df = df.merge(temp_df_r, how='left', on='grid_id')

In [48]:
df.head(2)

,grid_id,GridIdEx,1km2_ID,Easting,Northing,area,borough,zone,geometry,total_pollutant_co2_2019,...,total_road_length_a_road,total_road_length_b_road,total_road_length_c_unclassified_road,total_road_length_motorway,total_road_count_a_road,total_road_count_b_road,total_road_count_c_unclassified_road,total_road_count_motorway,mean_road_speed_non_bus,mean_road_speed_bus
0,1,1,5910,510500,203500,1.0,NonGLA,NonGLA,"POLYGON ((511000.001 203000.001, 509999.999 20...",1212.883859,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0
1,2,2,5911,511500,203500,1.0,NonGLA,NonGLA,"POLYGON ((512000.003 203000.001, 511000.001 20...",23432.691922,...,0.0,0.0,0.0,8143.46032,0.0,0.0,0.0,43.0,89.418605,0.0


## Encoding 

In [49]:
borough_encoded = pd.get_dummies(df["borough"], prefix="borough")
df = pd.concat([df, borough_encoded], axis=1)

df.drop("borough", axis=1, inplace=True)

df.head(2)

,grid_id,GridIdEx,1km2_ID,Easting,Northing,area,zone,geometry,total_pollutant_co2_2019,total_pollutant_pm10_2019,...,borough_Merton,borough_Newham,borough_NonGLA,borough_Redbridge,borough_Richmond,borough_Southwark,borough_Sutton,borough_Tower Hamlets,borough_Waltham Forest,borough_Wandsworth
0,1,1,5910,510500,203500,1.0,NonGLA,"POLYGON ((511000.001 203000.001, 509999.999 20...",1212.883859,0.162653,...,False,False,True,False,False,False,False,False,False,False
1,2,2,5911,511500,203500,1.0,NonGLA,"POLYGON ((512000.003 203000.001, 511000.001 20...",23432.691922,9.893812,...,False,False,True,False,False,False,False,False,False,False


In [50]:
zone_ordinal_mapping = {'NonGLA': 0, 'OuterULEX': 1, 'InnerULEX': 2, 'Central': 3}

df["zone"] = df["zone"].map(zone_ordinal_mapping)

In [51]:
df

,grid_id,GridIdEx,1km2_ID,Easting,Northing,area,zone,geometry,total_pollutant_co2_2019,total_pollutant_pm10_2019,...,borough_Merton,borough_Newham,borough_NonGLA,borough_Redbridge,borough_Richmond,borough_Southwark,borough_Sutton,borough_Tower Hamlets,borough_Waltham Forest,borough_Wandsworth
0,1,1,5910,510500,203500,1.00,0,"POLYGON ((511000.001 203000.001, 509999.999 20...",1212.883859,0.162653,...,False,False,True,False,False,False,False,False,False,False
1,2,2,5911,511500,203500,1.00,0,"POLYGON ((512000.003 203000.001, 511000.001 20...",23432.691922,9.893812,...,False,False,True,False,False,False,False,False,False,False
2,3,3,5912,512500,203500,1.00,0,"POLYGON ((512999.997 203000.001, 512000.003 20...",11388.566615,4.926730,...,False,False,True,False,False,False,False,False,False,False
3,4,4,5915,515500,203500,1.00,0,"POLYGON ((516000.004 203000.001, 515000.002 20...",2682.887565,0.485676,...,False,False,True,False,False,False,False,False,False,False
4,5,5,5916,516500,203500,1.00,0,"POLYGON ((516999.998 203000.001, 516000.004 20...",2353.626904,0.355503,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,3456,3351,10059,531500,179500,0.01,2,"POLYGON ((531999.998 179111.359, 531999.998 17...",1349.742960,0.523145,...,False,False,False,False,False,True,False,False,False,False
3456,3457,3352,10059,531500,179500,0.62,3,"POLYGON ((531999.998 179111.359, 531998.819 17...",19863.896057,5.594900,...,False,False,False,False,False,True,False,False,False,False
3457,3458,3353,9714,530500,181500,0.80,3,"POLYGON ((530999.996 181609.804, 530967.898 18...",52020.710502,14.513342,...,False,False,False,False,False,False,False,False,False,False
3458,3459,3354,9716,532500,181500,0.09,3,"MULTIPOLYGON (((532946.103 181894.902, 532939....",4474.873318,0.832513,...,False,False,False,False,False,False,False,False,False,False


In [52]:
df.drop(columns=['GridIdEx', '1km2_ID', 'Easting', 'Northing', 'geometry'], inplace=True)

In [53]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3460 entries, 0 to 3459
Data columns (total 95 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   grid_id                                3460 non-null   int32  
 1   area                                   3460 non-null   float64
 2   zone                                   3460 non-null   int64  
 3   total_pollutant_co2_2019               3460 non-null   float64
 4   total_pollutant_pm10_2019              3460 non-null   float64
 5   total_pollutant_pm2.5_2019             3460 non-null   float64
 6   total_pollutant_nox_2019               3460 non-null   float64
 7   total_rail_length                      3460 non-null   float64
 8   total_rail_stations                    3460 non-null   float64
 9   total_rail_link_terminations           3460 non-null   float64
 10  total_shipping_length                  3460 non-null   float64
 

In [54]:
df.to_excel("processed_data/final_df.xlsx", index=False)

`THE END`

---